# Clustering Crypto

In [3]:
# Initial imports
import pandas as pd
!pip install hvplot
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


Traceback (most recent call last):
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\Scripts\pip-script.py", line 6, in <module>
    from pip._internal.cli.main import main
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-packages\pip\_internal\cli\main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-packages\pip\_internal\cli\autocompletion.py", line 9, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-packages\pip\_internal\cli\main_parser.py", line 7, in <module>
    from pip._internal.cli import cmdoptions
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-packages\pip\_internal\cli\cmdoptions.py", line 24, in <module>
    from pip._internal.cli.progress_bars import BAR_TYPES
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-package

ModuleNotFoundError: No module named 'hvplot'

    return yield_lines(self.get_metadata(name))
  File "C:\Users\govind_patwal\Anaconda3\envs\PythonData\lib\site-packages\pip\_vendor\pkg_resources\__init__.py", line 1424, in get_metadata
    return value.decode('utf-8')
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 9840: invalid start byte in PKG-INFO file at path: c:\users\govind_patwal\anaconda3\envs\pythondata\lib\site-packages\holoviews-1.13.4-py3.7.egg-info\PKG-INFO


### Deliverable 1: Preprocessing the Data for PCA

In [ ]:
# Load the crypto_data.csv dataset.
path_of_file_to_read = 'crypto_data.csv'
crypto_df = pd.read_csv(path_of_file_to_read, index_col=0)
crypto_df

In [ ]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[(crypto_df['IsTrading'] == True)]
crypto_df

In [ ]:
# Keep all the cryptocurrencies that have a working algorithm.
crypto_df = crypto_df.loc[(crypto_df['Algorithm'] != 'NaN') | (crypto_df['Algorithm'] != '') ]
crypto_df

In [ ]:
# Remove the "IsTrading" column. 
crypto_df = crypto_df.drop(['IsTrading'], axis=1)
crypto_df

In [ ]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna()
crypto_df

In [ ]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[(crypto_df['TotalCoinsMined'] > 0)]
crypto_df

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
currencies_names_df = crypto_df[["CoinName"]]
currencies_names_df

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(['CoinName'], axis=1)
crypto_df

In [ ]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_df, columns = ['Algorithm','ProofType'])
X

In [ ]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)
X

### Deliverable 2: Reducing Data Dimensions Using PCA

In [ ]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)
crypto_pca

In [ ]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=crypto_pca,columns=["PC 1", "PC 2", "PC 3"], index = crypto_df.index
)
pcs_df

### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [ ]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


Running K-Means with `k=4`

In [ ]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)
predictions

In [ ]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df,pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = currencies_names_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    title="3D Scatter for K = 4",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"],
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'], sortable=True, selectable=True)

In [ ]:
# Print the total number of tradable cryptocurrencies.
Dataframe_length = len(clustered_df)
print(f'There are {Dataframe_length} tradable cryptocurrencies.')

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df_scaled = MinMaxScaler().fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
clustered_df_scaled

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=clustered_df_scaled, columns=["TotalCoinSupply", "TotalCoinsMined"], index = clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)